In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
dates = pd.date_range("2020-01-01", "2020-07-01")
data = pd.DataFrame()
for idate in tqdm(dates):
    url = f"https://archive.analytics.mybinder.org/events-{idate:%Y-%m-%d}.jsonl"
    idata = pd.read_json(url, lines=True)
    data = data.append(idata)

 92%|█████████▏| 169/183 [27:21<02:16,  9.74s/it]/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:7116: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
100%|██████████| 183/183 [29:45<00:00,  9.76s/it]


In [43]:
# Prune some columns
KEEP_COLS = ["timestamp", "provider", "spec", "origin"]
data = data[KEEP_COLS]

# Timestamp -> date
data['date'] = data['timestamp'].dt.date
data.drop('timestamp', axis=1, inplace=True)

In [56]:
# Set the timestamps to count number of launches by day
daily = (
    data.groupby(['spec', 'provider', 'origin', 'date']).agg({"spec": "count"}).
    rename(columns={'spec': 'launches'}).reset_index()
)

# Convert back to DT so we can split by month
daily['date'] = pd.to_datetime(daily['date'])

In [76]:
# Save to disk by month
for month, idata in daily.groupby(pd.Grouper(freq='M', key='date')):
    path_data = f"../data/{month:%Y-%m}.json"
    idata.to_json(path_data)